In [5]:
import cv2
import os
import imutils
import time
import dlib
import math
from scipy.spatial import distance as dist
import face_recognition
import numpy as np
from threading import Thread

In [6]:
def alarm(msg):
    global alarm_status
    global alarm_status2
    global saying

    while alarm_status:
        print('call')
        s = 'speak "'+msg+'"'
        os.system(s)

    if alarm_status2:
        print('call')
        saying = True
        s = 'speak "' + msg + '"'
        os.system(s)
        saying = False

In [7]:
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    C = dist.euclidean(eye[0], eye[3])

    ear = (A + B) / (2.0 * C)

    return ear

In [8]:
EYE_AR_THRESH = 0.2
EYE_AR_CONSEC_FRAMES = 20
YAWN_THRESH = 10

In [9]:
def main():
    COUNTER = 0
    alarm_status = False
    alarm_status2 = False
    saying = False
    video_capture = cv2.VideoCapture(0)

    ret, frame = video_capture.read(0)
    img = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    rgb_img = img[:, :, ::-1]

    face_landmarks_list = face_recognition.face_landmarks(rgb_img)
    
    process = True

    while True:
        ret, frame = video_capture.read(0)

        img = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        rgb_img = img[:, :, ::-1]
        
        if process:
            face_landmarks_list = face_recognition.face_landmarks(rgb_img)

            for face_landmark in face_landmarks_list:
                left_eye = face_landmark['left_eye']
                right_eye = face_landmark['right_eye']
                
                ear_left = eye_aspect_ratio(left_eye)
                ear_right = eye_aspect_ratio(right_eye)
                
                ear = (ear_left + ear_right)/2
                
                top_lip = face_landmark['top_lip']

                low_lip = face_landmark['bottom_lip']
                
                top_mean = np.mean(top_lip, axis=0)
                low_mean = np.mean(low_lip, axis=0)

                distance = abs(top_mean[1] - low_mean[1])


                color = (255,0,0)
                thickness = 4

#                 cv2.rectangle(frame, left_eye[0], right_eye[-1], color, thickness)
#                 cv2.rectangle(frame, top_lip[0], low_lip[-1], color, thickness)
                
                if ear < EYE_AR_THRESH:
                    COUNTER += 1

                    if COUNTER >= EYE_AR_CONSEC_FRAMES:
                        if alarm_status == False:
                            alarm_status = True
                            t = Thread(target=alarm, args=('wake up sir',))
                            t.deamon = True
                            t.start()
                        cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                else:
                    COUNTER = 0
                    alarm_status = False
                
                if (distance > YAWN_THRESH):
                        cv2.putText(frame, "Yawn Alert", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                        if alarm_status2 == False and saying == False:
                            alarm_status2 = True
                            t = Thread(target=alarm, args=('take some fresh air sir',))
                            t.deamon = True
                            t.start()
                        
                else:
                    alarm_status2 = False
                
                cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "YAWN: {:.2f}".format(distance), (300, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                
            cv2.imshow("Frame", frame)
                
            key = cv2.waitKey(1) & 0xFF

            if key == ord("q"):
                break
       
        process = not process

In [10]:
if __name__ == "__main__":
    main()


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/vishwajeet/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/vishwajeet/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-21ffbcac7263>", line 6, in alarm
    while alarm_status:
NameError: name 'alarm_status' is not defined

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/vishwajeet/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/vishwajeet/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-21ffbcac7263>", line 6, in alarm
    while alarm_status:
NameError: name 'alarm_status' is not defined

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/home/vishwajeet/anaconda3/lib/python3.7/threading.py", line 926, i